In [ ]:

import re
import requests
import threading
import pandas as pd

def fetch_data(page, cooperative_data):
    url = 'https://data.diskopukm.jatimprov.go.id/satu_data/webnya/ajax_listOds'
    payload = {
        'draw': 1,
        'columns[0][data]': '0',
        'columns[0][name]': '',
        'columns[0][searchable]': 'true',
        'columns[0][orderable]': 'true',
        'columns[0][search][value]': '',
        'columns[0][search][regex]': 'false',
        'order[0][column]': '0',
        'order[0][dir]': 'asc',  
        'start': page,
        'length': 10,
        'search[value]': '',
        'search[regex]': 'false',
    }
    response = requests.post(url, data=payload)
    if response.status_code == 200:
        data = response.json()
        cooperative_data.extend(data['data'])
    else:
        print("Failed to fetch data")

def extract_id_kop(text):
    match = re.search(r'(\d+)', text)
    if match:
        return match.group(1)
    else:
        return None

def main():
    cooperative_data = []
    threads = []
    num_data = 22039
    num_requests = num_data // 10 + (num_data % 10 > 0)
    for i in range(num_requests):
        page = i * 10
        thread = threading.Thread(target=fetch_data, args=(page, cooperative_data))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    column_titles = ['NO', 'ID KOP', 'KOPERASI', 'ALAMAT', 'NO BADAN HUKUM', 'TANGGAL BADAN HUKUM', 'DESA', 'KELURAHAN', 'AKSI']

    df = pd.DataFrame(cooperative_data, columns=column_titles)
    
    # Extract ID KOP from AKSI column
    df['ID KOP'] = df['AKSI'].apply(extract_id_kop)
    
    pd.set_option('display.max_colwidth', None)

    print(df.to_string(index=False))

if __name__ == "__main__":
    main()
